# Car-following model (Base code)

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the estimation of a GM car-following model in python with maximum likelihhod estimation:

* Load necessary packages
* Define variables and parameters to estimate
* Model specification
* Model output

Have a go at working through the notebook. To run a code cell, just click on it (to see a green box around it) and then press the **Run** button at the top! 

Some cells have blank lines for you to complete. There is always a comment telling you what to do!

You can also add a new cell by pressing the **+** button at the top of the page.

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [1]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
import csv # we need this to store our parameters as csv

### Let's give a name to our model

In [2]:
model_name = 'car_following_model25' # Name we want to give to our model (this is used when saving the output)

### Panel structure
We need to define whether our data is panel (i.e. multiple observations per individual) or not

In [3]:
panel = 1 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [4]:
mixing = 0 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [5]:
# Command to load the data
data = pd.read_table('data25.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we had opened our data set with a different name e.g. *database*

In [6]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)
data

,ID,Time,Position,Length,Width,Type,Speed,Acceleration,Lane,Leader,...,lead01,lead02,lead03,lead04,lane01,lane02,lane03,lane04,tasks,task_index
0,36,4,154.57,4.14,1.82,Car,15.03,0.24,7,54,...,1,1,1,NaN,1,1,1,NaN,14,1
1,36,5,169.77,4.14,1.82,Car,15.17,0.05,7,54,...,1,1,1,1.0,1,1,1,1.0,14,2
2,36,6,184.91,4.14,1.82,Car,15.21,0.05,7,54,...,1,1,1,1.0,1,1,1,1.0,14,3
3,36,7,200.12,4.14,1.82,Car,15.30,0.13,7,54,...,1,1,1,1.0,1,1,1,1.0,14,4
4,36,8,215.52,4.14,1.82,Car,15.45,0.16,7,54,...,1,1,1,1.0,1,1,1,1.0,14,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9086,2821,1173,102.54,4.45,1.82,Car,6.97,-0.08,7,2817,...,1,1,1,1.0,1,1,1,1.0,13,9
9087,2821,1174,109.45,4.45,1.82,Car,6.88,-0.11,7,2817,...,1,1,1,1.0,1,1,1,1.0,13,10
9088,2821,1175,116.32,4.45,1.82,Car,6.75,-0.14,7,2817,...,1,1,1,1.0,1,1,1,1.0,13,11
9089,2821,1176,122.97,4.45,1.82,Car,6.62,-0.08,7,2814,...,0,0,0,0.0,1,1,1,1.0,13,12


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [7]:
print(data.columns)

Index(['ID', 'Time', 'Position', 'Length', 'Width', 'Type', 'Speed',
       'Acceleration', 'Lane', 'Leader', 'Follower', 'Space_headway',
       'Time_headway', 'Acceleration_lead', 'Speed_lead', 'Position_lead',
       'Density', 'distanceToEnd', 'Speed_test', 'Space_headway1',
       'Time_headway1', 'lane_change', 'lead_change', 'lag_leader0',
       'lag_leader1', 'lag_leader2', 'lag_leader3', 'lag_leader4', 'lag_lane0',
       'lag_lane1', 'lag_lane2', 'lag_lane3', 'lag_lane4', 'lag_acceleration0',
       'lag_acceleration1', 'lag_acceleration2', 'lag_acceleration3',
       'lag_acceleration4', 'lag_acceleration_lead0', 'lag_acceleration_lead1',
       'lag_acceleration_lead2', 'lag_acceleration_lead3',
       'lag_acceleration_lead4', 'lag_speed0', 'lag_speed1', 'lag_speed2',
       'lag_speed3', 'lag_speed4', 'lag_speed_lead0', 'lag_speed_lead1',
       'lag_speed_lead2', 'lag_speed_lead3', 'lag_speed_lead4',
       'lag_position_lead0', 'lag_position_lead1', 'lag_position_lead

## Data processing

* It may be the case that our data do not include all variables required for modelling.
* In this case we must generate the additional variables that we need

### Variable creation

* Let's create a "running task" variable, else a counter for the observations of each driver

In [8]:
data['running_task'] = data.groupby(['ID']).cumcount()+1 # counter of observations per individual

## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.

In [9]:
# Example:
# Variable_name = np.array(data['Variable_name']).reshape(-1, 1)

running_task = np.array(data['running_task']).reshape(-1, 1)
Acceleration = np.array(data['Acceleration']).reshape(-1, 1)
Speed = np.array(data['Speed']).reshape(-1, 1)
Space_headway = np.array(data['Space_headway']).reshape(-1, 1)
Speed_lead = np.array(data['Speed_lead']).reshape(-1, 1)
Acceleration_lead = np.array(data['Acceleration_lead']).reshape(-1, 1)

#### Define the ID variable

In [10]:
ID = np.array(data['ID']) # ID does not need to be reshaped

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated.

* The starting values are usually zeroes
* When a parameter is included as a denominator, the starting value cannot be 0 for computational reasons.
* However, since we estimate the log of sigma, our starting value can be zero since exp(sigma) can never be absolute zero.

In [11]:
betas_start = {
               "alpha_acc":0, # Acceleration constant parameter
               "alpha_dec":0, # Deceleration constant parameter
               "beta_acc":0,  # Speed (acceleration regime) parameter
               "beta_dec":0,  # Speed (deceleration regime) parameter
               "gamma_acc":0, # Space headway (acceleration regime) parameter
               "gamma_dec":0, # Space headway (deceleration regime) parameter
               "lamda_acc":0, # Relative speed (acceleration regime) parameter
               "lamda_dec":0, # Relative speed (deceleration regime) parameter
               "sigma_acc":0, # Std deviation (acceleration regime) parameter
               "sigma_dec":0  # Std deviation (deceleration regime) parameter
                }

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [12]:
# ### Activate this cell to load old results ###

# # Open old iteration file
# betas_old = pd.read_csv('model_name_iterations.csv')

# # Keep last row
# betas_old = betas_old.tail(1)

# # Convert to dictionary
# betas_old = betas_old.to_dict(orient='records')[0]

# # Copy values from old to start for keys that are common to both dictionaries
# betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# # Delete old estimates
# del[betas_old]

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [13]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]

    # Car-following model specification

    # BE CAREFUL!! Below we add a small correction term (np.exp(-50)) to speed and relative speed.
    # This correction is included to avoid estimation issues if the value of the independent variables is 0.
    
    # Sensitivity term
    sensitivity_acc = alpha_acc*((Speed+np.exp(-50))**beta_acc)/(Space_headway**gamma_acc)
    sensitivity_dec = alpha_dec*((Speed+np.exp(-50))**beta_dec)/(Space_headway**gamma_dec)

    # Stimulus term
    stimulus_acc = np.abs(Speed_lead - Speed + np.exp(-50))**lamda_acc
    stimulus_dec = np.abs(Speed_lead - Speed + np.exp(-50))**lamda_dec

    # Acceleration - deceleration means
    acc = sensitivity_acc*stimulus_acc
    dec = sensitivity_dec*stimulus_dec

    # Density functions for acceleration and deceleration
    pdf_acc = norm.pdf(Acceleration,acc,np.exp(sigma_acc))
    pdf_dec = norm.pdf(Acceleration,dec,np.exp(sigma_dec))
    
    # Combined probability of acceleration and deceleration regimes
    P = pdf_acc*(Speed_lead - Speed>=0)+pdf_dec*(Speed_lead - Speed<0)
    
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)
    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [14]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [15]:
import warnings
from scipy.stats import t
# with warnings.catch_warnings():
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**It is advisable not to edit the lines of code below**

In [16]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'BFGS' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: [-9292.75705536]
Current log likelihood: [-5060.57474182]
Current log likelihood: [-4847.97000976]
Current log likelihood: [-4751.11579706]
Current log likelihood: [-4569.43384532]
Current log likelihood: [-4540.29782087]
Current log likelihood: [-4490.16321991]
Current log likelihood: [-4470.13273638]
Current log likelihood: [-4455.95803459]
Current log likelihood: [-4431.19511565]
Current log likelihood: [-4233.89695643]
Current log likelihood: [-4116.91934835]
Current log likelihood: [-4072.19351353]
Current log likelihood: [-4043.89121771]
Current log likelihood: [-4035.18155266]
Current log likelihood: [-4021.82482432]
Current log likelihood: [-4002.61185619]
Current log likelihood: [-3982.37978427]
Current log likelihood: [-3967.18123569]
Current log likelihood: [-3946.76225348]
Current log likelihood: [-3912.85539201]
Current log likelihood: [-3866.37738701]
Current log likelihood: [-3849.1635473]
Current log likelihood: [-3821.88109052]
Current log likel

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [17]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL,method='forward')
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL,method='forward')
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)
... Done!!


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to mis-specified models.

In [18]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric,errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

,Parameter,Estimate,s.e.,t-ratio0,p-value,Rob s.e.,Rob t-ratio0,Rob p-value
0,alpha_acc,0.159,0.041,3.880,0.000,0.117,1.361,0.173
1,alpha_dec,-5.912,0.534,-11.065,0.000,1.939,-3.050,0.002
2,beta_acc,0.110,0.095,1.156,0.248,0.276,0.397,0.691
3,beta_dec,-0.373,0.040,-9.308,0.000,0.101,-3.680,0.000
4,gamma_acc,0.049,0.056,0.874,0.382,0.097,0.504,0.614
5,gamma_dec,0.612,0.030,20.519,0.000,0.129,4.755,0.000
6,lamda_acc,0.727,0.046,15.664,0.000,0.096,7.608,0.000
7,lamda_dec,0.806,0.024,33.414,0.000,0.097,8.319,0.000
8,sigma_acc,-1.120,0.011,-106.279,0.000,0.033,-33.643,0.000
9,sigma_dec,-0.899,0.010,-86.134,0.000,0.035,-25.700,0.000


## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [19]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

Log likelihood at zeros:[-9292.75705536]
Initial log likelihood:[-9292.75705536]
Final log likelihood:-3731.192739805429
rho squared=[0.59848377]
adjusted rho squared=[0.59740767]
AIC=7482.385479610858
BIC=7553.535881532077


## Save output

We can save our output using the code below (do not edit):

In [20]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)